# SignLanguageDetection_EmbeddedMachineLearning-RaspberryPi

Project to deploy a Machine Learning Model on a Rasberry Pi.  
Training a model to identify american sign language letters in images.

- First Time setup  
  - To download the dataset and the pre-trained-model

## References

### Training the model
1. Following [*Training Custom Object Detector*](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html) tutorial
     - [TensorFlow Installation](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html#tf-install) Notes:
       - With Anaconda installed, use `conda info --envs` to see created environments
       - Before working on the project, do `conda activate signLanguageDetector` to activate the environment every time
       - Install cuda and cudnn tu use GPU on manjaro with `sudo pacman -S cuda cudnn`
       - For Object detection API, added the [TensorFlow Models repository](https://github.com/tensorflow/models) as a git submodule with the command `git submodule add https://github.com/tensorflow/models ./TensorFlow/models/`
       - To test if everything worked out in the installation you can run `python TensorFlow/models/research/object_detection/builders/model_builder_tf2_test.py`
      - Dataset from roboflow.com: [American Sign Language Letters Dataset](https://public.roboflow.com/object-detection/american-sign-language-letters/1)
      - Had to do [this](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2.md#python-package-installation) before training to fix an error